## 새글 알림봇 만들기

1. 웹페이지 가져와서 URL 추출하기 - requests, BeautifulSoup 라이브러리
2. URL들이 기존에 것과 있는지 확인하기 - 파이썬 표준 라이브러리
3. 새 URL이면 텔레그램으로 전송하기 - python-telegram-bot 라이브러리

In [ ]:
#기본 함수 만들고 호출해보기
def test_func():
    print("hello, world")

In [ ]:
test_func()

In [ ]:
#파일쓰기
with open("coindesk.txt", 'w', encoding='utf8') as f:
    f.write("http://test_url_1.com" + '\n')
    f.write("http://test_url_2.com" + '\n')

In [ ]:
#파일에 한줄 추가하기
with open("coindesk.txt", 'a', encoding='utf8') as f:
    f.write("http://test_url_3.com" + '\n')

In [ ]:
#파일읽기
with open("coindesk.txt", 'r', encoding='utf8') as f:
    text = f.read()

In [ ]:
#파일로 부터 읽은 text를 리스트로 만들어보기
url_list = text.split('\n')
url_list

In [ ]:
#리스트에 해당 text가 있는지 없는지를 bool(True / False) 확인 문법 'in'
'http://test_url_1.com' in url_list

In [ ]:
#위의 과정을 updator 함수로 만들어보기
def updator(new_url):
    with open("coindesk.txt", 'r', encoding='utf8') as f:
        text = f.read()
    
    old_url_list = text.split('\n')
    
    if new_url in old_url_list:
        print("EXIST URL")
    else:
        with open("coindesk.txt", 'a', encoding='utf8') as f:
            f.write(new_url + '\n')
                    
        print("NEW_URL")

In [ ]:
updator("http://test_url_4.com") #새로운 글이라는 것을 확인

In [ ]:
updator("http://test_url_4.com") #2번째 시도에는 이미 http://test_url_4.com 이 기록되어 있음

In [ ]:
#구글 blockchain 뉴스 검색 url
URL = "https://news.google.com/news/headlines/section/q/blockchain/blockchain?ned=kr&hl=ko"

In [ ]:
# requests 와 BeautifulSoup라이브러리 호출
import requests
from bs4 import BeautifulSoup as BS

In [ ]:
resp = requests.get(URL)
resp

In [ ]:
#해당 url로 보낸응답 text를 beautifulsoup객체로 변환
bs_html = BS(resp.text, "html.parser")

In [ ]:
type(bs_html)

In [ ]:
# 해당 뉴스 html 에 공통적으로 c-wiz가 포함되어 있음, 물론 뉴스 이외에 것들도 있다.
bs_wiz = bs_html.find_all('c-wiz')

In [ ]:
len(bs_wiz)

In [ ]:
# 링크가 들어 있는 태그는 c-wiz안에 들어 있는 a 태그이며, a 태그 안에 href로 존재한다.
bs_wiz[23].next_element.div.a.get('href')

In [ ]:
# c-wiz중에 a 태그가 없는것들도 있다. try~except으로 예외처리 해준다.
bs_wiz[29].next_element.div.a.get('href') 

In [ ]:
#위의 과정을 하나로 해서 resp.text에 뉴스글에 해당하는 url들을 all_url 리스트에 하나씩 추가해본다.
all_url = [] # 결과를 위한 list선언
bs_wiz = bs_html.find_all('c-wiz')

for elem in bs_wiz: #bs_wiz를 순환하면서 href를 가져오는 과정
    try:#try구문 안에 위치하고 있어 실패시(에러) 아무런 동작을 하지 않는다.
        elem_url = elem.next_element.div.a.get('href') 
        if str(elem_url).startswith('http'): #http로시작하지 않는 것들은 추가하지 않는다.
            all_url.append(elem_url)
    except Exception as e:
        pass

In [ ]:
all_url #결과를 보면 중복되는 것들이 있다.

In [ ]:
#위의 과정을 함수로 만들어본다.
URL = "https://news.google.com/news/headlines/section/q/blockchain/blockchain?ned=kr&hl=ko"

def loader():#입력해주는 변수가 필요 없다.
    resp = requests.get(URL)
    bs_html = BS(resp.text, "html.parser")
    
    all_url = []

    bs_wiz = bs_html.find_all('c-wiz')
    
    for elem in bs_wiz:
        try:
            elem_url = elem.next_element.div.a.get('href')
        except Exception as e:
            pass
    
    result_url = [] #최종 반환되는 리스트 추가
    
    for url in all_url:
        if url[:4] == 'http':
            result_url.append(url)
    
    # for url in all_url ~ 루프는 아래 list comprehention이라는 파이썬 문법으로도 가능하다.
    #result_url = [url for url in all_url in url[:4] == 'http']
    
    return list(set(result_url))

In [ ]:
#텔레그램으로 메시지 보내기
import telegram

In [ ]:
#필요한 부분은 Bot Father로 부터 받은 TOKEN 키와 해당 봇과 대화중인 chat_id이다.
TOKEN = "Telegram_bot_api_key"
CHAT_ID = "your_chat_id"

In [ ]:
bot = telegram.Bot(TOKEN)

In [ ]:
#chat_id를 얻으려면 해당 봇으로 말을 건후(개인 talk) 응담으로 getUpdates로 수신된 메시지를 확인한다
updates = bot.getUpdates()

In [ ]:
print(updates[0])

In [ ]:
bot.send_message(CHAT_ID, text="*반가워요*", parse_mode='Markdown')

In [ ]:
#역시 위의 과정을 함수로 만들어 본다.
def send_to_telegram(text_message):
    bot = telegram.Bot(TOKEN)
    bot.send_message(CHAT_ID, text_message)
    print("sent msg")

In [ ]:
# 시간 인터벌을 주기 위한 time
import time

In [ ]:
time.sleep(10)
print("after 10 second, hello")

In [ ]:
URL = "https://news.google.com/news/headlines/section/q/blockchain/blockchain?ned=kr&hl=ko"

def loader():
    resp = requests.get(URL)
    bs_html = BS(resp.text, "html.parser")
    
    all_url = []
    bs_wiz = bs_html.find_all('c-wiz')
    
    for elem in bs_wiz:
        try:
            elem_url = elem.next_element.div.a.get('href')
            if str(elem_url).startswith('http'):
                all_url.append(elem_url)
        except Exception as e:
            pass
    
    return list(set(all_url))

In [ ]:
def updater(new_url):
    with open("coindesk.txt", 'r', encoding='utf8') as f:
        text = f.read()
    
    old_url_list = text.split('\n')
    
    if new_url in old_url_list:
        print("EXIST URL")
        return False
    else:
        with open("coindesk.txt", 'a', encoding='utf8') as f:
            f.write(new_url + '\n')
                    
        print("NEW_URL")
        return True

In [ ]:
def send_to_telegram(text_message):
    bot = telegram.Bot(TOKEN)
    bot.send_message(CHAT_ID, text_message)
    print("sent msg")

In [ ]:
#'loader' - 'updater' - 'send_to_telegram' 순으로 무한 loop로 해당 과정을 계속한다.
while True:
    all_url = loader()
    for url in all_url:
        if updater(url): #수정된 updater는 기존에 없던 url인 경우 True를 반환한다.
            send_to_telegram(url)
            
    print("one cycle complete")        
    time.sleep(10)